In [1]:
import tensorflow as tf
from tensorflow import keras
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 1. #what portion of gpu to use
session = tf.Session(config=config)
keras.backend.set_session(session)

In [2]:
from __future__ import print_function, division

from keras.datasets import mnist
from keras.layers import Input, Dense, Reshape, Flatten, Dropout, multiply, Add, Lambda, add
from keras.layers import BatchNormalization, Activation, Embedding, ZeroPadding2D, concatenate
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D, Conv2DTranspose, MaxPooling2D
from keras.models import Sequential, Model
from keras.optimizers import Adam, RMSprop
from tensorflow.examples.tutorials.mnist import input_data
from skimage.io import imread
from keras.utils import to_categorical

import matplotlib.pyplot as plt

import numpy as np

Using TensorFlow backend.


In [3]:
from pathlib import Path

classes = ['BUBBLES_v3','CLEAR_BLOB','DILATED','TURBID_v2','UNDEFINED','WALL','WRINKLES_v2']
#classes = ['BUBBLES_v3','CLEAR_BLOB','DILATED','UNDEFINED','WALL','WRINKLES_v2']
mainDir = '/data/intestins/original/'

In [4]:
img_per_class = 1000
img_per_test = 100
cls = 0
X_train = np.empty([7*img_per_class,256,256,3])
y_train = np.zeros([7*img_per_class])
X_test = np.empty([7*img_per_test,256,256,3])
y_test = np.zeros([7*img_per_test])
for c in classes:
    cnt = 0
    curDir = mainDir + c
    pathlist = Path(curDir).glob('*')
    for path in pathlist:
        path_str = str(path)
        if "png" not in path_str and "jpg" not in path_str and "bmp" not in path_str:
            continue
        if "r180" in path_str or "r90" in path_str:
            continue
        im = imread(path_str)
        imarray = np.array(im)
        imresized = resize(imarray, (256,256), mode='symmetric')
        if cnt >= img_per_class + img_per_test:
            break
        elif cnt >= img_per_class:
            X_test[cls * img_per_test + cnt - img_per_class] = imresized
            y_test[cls * img_per_test + cnt - img_per_class] = cls
        else:
            X_train[cls * img_per_class + cnt] = imresized
            y_train[cls * img_per_class + cnt] = cls
        cnt += 1
    cls += 1

In [ ]:
#Based on code by Rowel Atienza (https://github.com/roatienza/Deep-Learning-Experiments)

import numpy as np
import time
import tensorflow as tf
from tensorflow import keras
from tensorflow.examples.tutorials.mnist import input_data
import math

from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten, Reshape
from keras.layers import Conv2D, Conv2DTranspose, UpSampling2D
from keras.layers import LeakyReLU, Dropout
from keras.layers import BatchNormalization
from keras.optimizers import Adam, RMSprop

import matplotlib.pyplot as plt

#import os
#os.environ["CUDA_VISIBLE_DEVICES"]="0"

#config = tf.ConfigProto()
#config.gpu_options.per_process_gpu_memory_fraction = 0.4 #what portion of gpu to use
#session = tf.Session(config=config)
#keras.backend.set_session(session)

class ElapsedTimer(object):
    def __init__(self):
        self.start_time = time.time()
    def elapsed(self,sec):
        if sec < 60:
            return str(sec) + " sec"
        elif sec < (60 * 60):
            return str(sec / 60) + " min"
        else:
            return str(sec / (60 * 60)) + " hr"
    def elapsed_time(self):
        print("Elapsed: %s " % self.elapsed(time.time() - self.start_time) )

class DCGAN(object):
    def __init__(self, img_rows=64, img_cols=64, channel=3):

        self.img_rows = img_rows
        self.img_cols = img_cols
        self.channel = channel
        self.img_shape = (self.img_rows, self.img_cols, self.channel)
        self.num_classes = 7
        self.D = None   # discriminator
        self.G = None   # generator
        self.AM = None  # adversarial model
        self.DM = None  # discriminator model

    # (W−F+2P)/S+1
    def discriminator(self):
        
        depth = 64
        dropout = 0.2
        
        img = Input(shape=self.img_shape)
        
        label = Input(shape=(1,))
        label_embedding = Flatten()(Embedding(self.nclasses, self.latent_dim)(label))
        d1 = Dense(512, activation="relu")(label_embedding)
        d2 = Dense(4096, activation="relu")(d1)
        label_img_shape = Reshape((64,64,1))(d2)
        
        new_img = concatenate([img, label_img_shape], axis=3)
        
        c1 = Conv2D(depth, 5, strides=2, padding='same')(new_img)
        c1 = LeakyReLU(alpha=0.2)(c1)
        c1 = Dropout(dropout)(c1) # 32 x 32 x 64
        
        c2 = Conv2D(depth, 5, padding='same')(c1)
        c2 = LeakyReLU(alpha=0.2)(c2)
        c2 = Dropout(dropout)(c2) # 32 x 32 x 64
        
        c3 = Conv2D(depth, 5, padding='same')(c2)
        c3 = LeakyReLU(alpha=0.2)(c3)
        c3 = Dropout(dropout)(c3) # 32 x 32 x 64
        
        c4 = Conv2D(depth, 5, padding='same')(c3)
        c4 = LeakyReLU(alpha=0.2)(c4)
        c4 = Dropout(dropout)(c4) # 32 x 32 x 64
        
        a1 = Add()([c1, c4])
           
        c5 = Conv2D(depth*2, 5, strides=2, padding='same')(a1)
        c5 = LeakyReLU(alpha=0.2)(c5)
        c5 = Dropout(dropout)(c5) # 16 x 16 x 128
        
        c6 = Conv2D(depth*2, 5, padding='same')(c5)
        c6 = LeakyReLU(alpha=0.2)(c6)
        c6 = Dropout(dropout)(c6) # 16 x 16 x 128
        
        c7 = Conv2D(depth*2, 5, padding='same')(c6)
        c7 = LeakyReLU(alpha=0.2)(c7)
        c7 = Dropout(dropout)(c7) # 16 x 16 x 128
        
        c8 = Conv2D(depth*2, 5, padding='same')(c7)
        c8 = LeakyReLU(alpha=0.2)(c8)
        c8 = Dropout(dropout)(c8) # 16 x 16 x 128
        
        a2 = Add()([c5, c8])
        
        c9 = Conv2D(depth*4, 5, strides=2, padding='same')(a2)
        c9 = LeakyReLU(alpha=0.2)(c9)
        c9 = Dropout(dropout)(c9) # 8 x 8 x 256
        
        c10 = Conv2D(depth*4, 5, padding='same')(c9)
        c10 = LeakyReLU(alpha=0.2)(c10)
        c10 = Dropout(dropout)(c10) # 8 x 8 x 256
        
        c11 = Conv2D(depth*4, 5, padding='same')(c10)
        c11 = LeakyReLU(alpha=0.2)(c11)
        c11 = Dropout(dropout)(c11) # 8 x 8 x 256
        
        c12 = Conv2D(depth*4, 5, padding='same')(c11)
        c12 = LeakyReLU(alpha=0.2)(c12)
        c12 = Dropout(dropout)(c12) # 8 x 8 x 256
        
        a3 = Add()([c9, c12])
        
        c13 = Conv2D(depth*8, 5, strides=2, padding='same')(a3)
        c13 = LeakyReLU(alpha=0.2)(c13) # 4 x 4 x 512

        f = Flatten()(c13)
        
        validity = Dense(1)(f)
        
        self.D = Model(inputs=[img,label], output=validity)
        self.D.summary()
        return self.D
        
    
    def generator(self):
        
        dropout = 0.4
        depth = 256
        dim = 8
        
        noise = Input(shape=(self.latent_dim,))    
        label = Input(shape=(1,))   
        label_embedding = Flatten()(Embedding(self.nclasses, self.latent_dim)(label))
        model_input = concatenate([noise, label_embedding])
        
        # In: 100
        # Out: dim x dim x depth
        d = Dense(dim*dim*depth, activation="relu")(model_input)
        d = BatchNormalization(momentum=0.9)(d)
        d = Reshape((dim, dim, depth))(d) # 8 x 8 x 256

        # In: dim x dim x depth
        # Out: 2*dim x 2*dim x depth/2
        u1 = UpSampling2D()(d) # 16 x 16 x 256
        
        c1 = Conv2DTranspose(int(depth/2), 5, padding='same', activation="relu")(u1) # 16 x 16 x 128
        c1 = BatchNormalization(momentum=0.9)(c1)
        
        c2 = Conv2DTranspose(int(depth/2), 5, padding='same', activation="relu")(c1) # 16 x 16 x 128
        c2 = BatchNormalization(momentum=0.9)(c2)
        
        c3 = Conv2DTranspose(int(depth/2), 5, padding='same', activation="relu")(c2) # 16 x 16 x 128
        c3 = BatchNormalization(momentum=0.9)(c3)
        
        c4 = Conv2DTranspose(int(depth/2), 5, padding='same', activation="relu")(c3) # 16 x 16 x 128
        c4 = BatchNormalization(momentum=0.9)(c4)
        
        a1 = Add()([c1, c4])
        u2 = UpSampling2D()(a1) # 32 x 32 x 128
        
        c5 = Conv2DTranspose(int(depth/4), 5, padding='same', activation="relu")(u2) # 32 x 32 x 64
        c5 = BatchNormalization(momentum=0.9)(c5)
        
        c6 = Conv2DTranspose(int(depth/4), 5, padding='same', activation="relu")(c5) # 32 x 32 x 64
        c6 = BatchNormalization(momentum=0.9)(c6)
        
        c7 = Conv2DTranspose(int(depth/4), 5, padding='same', activation="relu")(c6) # 32 x 32 x 64
        c7 = BatchNormalization(momentum=0.9)(c7)
        
        c8 = Conv2DTranspose(int(depth/4), 5, padding='same', activation="relu")(c7) # 32 x 32 x 64
        c8 = BatchNormalization(momentum=0.9)(c8)
        
        a2 = Add()([c5, c8])
        u3 = UpSampling2D()(a2) # 64 x 64 x 64
        
        c9 = Conv2DTranspose(int(depth/8), 5, padding='same', activation="relu")(u3) # 64 x 64 x 32
        c9 = BatchNormalization(momentum=0.9)(c9)
        
        c10 = Conv2DTranspose(int(depth/8), 5, padding='same', activation="relu")(c9) # 64 x 64 x 32
        c10 = BatchNormalization(momentum=0.9)(c10)
        
        c11 = Conv2DTranspose(int(depth/8), 5, padding='same', activation="relu")(c10) # 64 x 64 x 32
        c11 = BatchNormalization(momentum=0.9)(c11)
        
        c12 = Conv2DTranspose(int(depth/8), 5, padding='same', activation="relu")(c11) # 64 x 64 x 32
        c12 = BatchNormalization(momentum=0.9)(c12)
        
        a3 = Add()([c9, c12])

        # Out: 28 x 28 x 1 grayscale image [0.0,1.0] per pix
        generated_img = Conv2DTranspose(3, 5, padding='same')(a3) # 64 x 64 x 3
        
        self.G = Model(inputs = [noise, label], output = generated_img) 
        self.G.summary()
        return self.G 

    def discriminator_model(self):
        if self.DM:
            return self.DM
        
        optimizer = RMSprop(lr=0.00005)
        self.DM = self.discriminator()
        self.DM.compile(loss=wasserstein_loss, optimizer=optimizer, metrics=['accuracy'])
        
        return self.DM

    def adversarial_model(self):
        if self.AM:
            return self.AM
        
        optimizer = RMSprop(lr=0.00005)
        
        D = self.discriminator()
        G = self.generator()
        
        noise = Input(shape=(100,))
        label = Input(shape=(1,))
        img = G([noise, label])
        valid = D([img, label])
        
        self.AM = Model([noise, label], valid)
        self.AM.compile(loss=wasserstein_loss, optimizer=optimizer)
        
        #self.AM.compile(optimizer='RMSprop', loss=custom_loss)
        return self.AM

class MNIST_DCGAN(object):
    def __init__(self, x_train, y_train):
        self.img_rows = 64
        self.img_cols = 64
        self.channel = 3
        
        self.x_train = x_train
        self.y_train = y_train

        self.DCGAN = DCGAN()
        self.discriminator =  self.DCGAN.discriminator_model()
        self.adversarial = self.DCGAN.adversarial_model()
        self.generator = self.DCGAN.generator()

        
    def train(self, train_steps=2000, batch_size=256, save_interval=0):
        
        history_d = []
        history_a = []
        history_acc = []
        
        noise_input = None
        if save_interval>0:
            noise_input = np.random.uniform(-1.0, 1.0, size=[35, 100])
            
        for epoch in range(train_steps):    

            idx = np.random.permutation(self.x_train.shape[0])
            X_train = self.x_train[idx]
            y_train = self.y_train[idx]
            
            steps = math.ceil(X_train.shape[0]/batch_size)
            
            for i in range(steps):
                
                if (i%3 == 2):
                    
                    history_d.append(d_loss[0])
                    history_acc.append(d_loss[1])
                    
                    # freeze D
                    self.discriminator.trainable = False
                    for l in self.discriminator.layers:
                        l.trainable = False

                    # train adverserial model
                    y = np.ones([batch_size, 1])
                    noise = np.random.uniform(-1.0, 1.0, size=[batch_size, 100])
                    sample_labels = np.random.randint(0, 7, batch_size).reshape(-1, 1)

                    a_loss = self.adversarial.train_on_batch([noise, sample_labels], y)
                    history_a.append(a_loss)
                    
                    # unfreeze D
                    self.discriminator.trainable = True
                    for l in self.discriminator.layers:
                        l.trainable = True
                        

                # clip D weights
                for l in self.discriminator.layers:
                    weights = l.get_weights()
                    weights = [np.clip(w, -0.01, 0.01) for w in weights]
                    l.set_weights(weights)
            
                images_train = X_train[i*batch_size:min((i+1)*batch_size,X_train.shape[0])]
                labels = y_train[i*batch_size:min((i+1)*batch_size,X_train.shape[0])]
                
                cur_batch_size = images_train.shape[0]
                
                noise = np.random.uniform(-1.0, 1.0, size=[cur_batch_size, 100])
                images_fake = self.generator.predict([noise, labels])
                
                x = np.concatenate((images_train, images_fake))
                ls = np.concatenate([labels, labels])
                y = np.ones([2*cur_batch_size, 1])
                y[cur_batch_size:, :] = -1
                
                d_loss = self.discriminator.train_on_batch([x, ls], y)       
            
            log_mesg = "%d: [D loss: %f, acc: %f]" % (epoch, d_loss[0], d_loss[1])
            log_mesg = "%s  [A loss: %f]" % (log_mesg, a_loss)
            print(log_mesg)
            
            if save_interval>0:
                if (epoch+1)%save_interval==0:
                    self.plot_images(save2file=True, samples=noise_input.shape[0],\
                        noise=noise_input, step=(epoch+1))
                    
        return history_d, history_a, history_acc


    def plot_images(self, save2file=False, fake=True, samples=35, noise=None, step=0):
        filename = 'intestines_cgan/mnist.png'
        if fake:
            if noise is None:
                noise = np.random.uniform(-1.0, 1.0, size=[samples, 100])
            else:
                filename = "intestines_cgan/mnist_%d.png" % step
            sample_labels = np.repeat(np.arange(7),samples/7)
            images = self.generator.predict([noise, sample_labels])
        else:
            i = np.random.randint(0, self.x_train.shape[0], samples)
            images = self.x_train[i, :, :, :]

        plt.figure(figsize=(12,15))
        for i in range(images.shape[0]):
            plt.subplot(7, samples/7, i+1)
            image = images[i, :, :, :]
            image = np.reshape(image, [self.img_rows, self.img_cols, self.channel])
            plt.imshow(image)
            plt.axis('off')
        plt.tight_layout()
        if save2file:
            plt.savefig(filename)
            plt.close('all')
        else:
            plt.show()

if __name__ == '__main__':
    mnist_dcgan = MNIST_DCGAN(X_train, y_train)
    timer = ElapsedTimer()
    history_d, history_a, history_acc = mnist_dcgan.train(train_steps=1000, batch_size=250, save_interval=1)
    timer.elapsed_time()
    mnist_dcgan.plot_images(fake=True)
    mnist_dcgan.plot_images(fake=False, save2file=True)